In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from azure.storage.blob import BlobServiceClient
import jupytext

subscription_id = '96599b9b-0d9b-4577-9163-418d2c6cd411'
resource_group = 'rg-datasci-ml-dev-001'
workspace_name = 'ml-moss-dev-001'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset_hourly_compare_cmic = Dataset.get_by_name(workspace, name='Hourly_Employee_Time_prod')
cmic_pr_df_dev = dataset_hourly_compare_cmic.to_pandas_dataframe()

dataset_hourly_compare_wd = Dataset.get_by_name(workspace, name='WD_Hourly_Employee_Time_prod')
wd_pr_df_dev = dataset_hourly_compare_wd.to_pandas_dataframe()

tcp_dataset = Dataset.get_by_name(workspace, name='WorkSegments_Timesheet_prod')
tcp_df_dev = tcp_dataset.to_pandas_dataframe()

jcjob_dataset = Dataset.get_by_name(workspace, name='JCJOB_TABLE_TN_prod')
jcjob_df = jcjob_dataset.to_pandas_dataframe()

In [28]:
##########################

week_ending = '2023-10-21'

##########################

date = datetime.strptime(week_ending, "%Y-%m-%d")
start_date = date - timedelta(days = 6)
end_date = date

In [70]:
tcp_df = tcp_df_dev.copy()

In [71]:
###TCP Data Manipulation
#Filter: Solar Jobs only 
#tcp_df = tcp_df.loc[tcp_df['Job_Code'].str.startswith('710')]
#Filter: Only Keeping Labor Hourly
tcp_df = tcp_df.loc[tcp_df['Task'].isin(['LH','LHA'])]
#Filter: Only keeping Moss Labor Hourly - excluding subs
tcp_df = tcp_df.loc[tcp_df['Department']=='710']
#Creating a total hours column
tcp_df['total_hours'] = tcp_df['Regular_Hours'] + tcp_df['Overtime_Hours']
#Renaming Relevant Columns
tcp_df = tcp_df.rename({'Employee_Number':'EEID','Job_Code':'Job','Activity':'Sub_Job','total_hours':'Hours'},axis='columns')
tcp_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
tcp_df_comp = tcp_df.loc[:,tcp_cols_keep]

In [73]:
wd_pr_df = wd_pr_df_dev.copy()

In [74]:
#Workday Data Manipulation
#DUPLICATE FILTER - from WQL Query for Reported time Blocks for a Worker
wql_filter = (wd_pr_df['timeEntryCode.id1'].notna())&(wd_pr_df['historicalStatus']!='94e8c24204d1406897f2758bd5e306d1')&(wd_pr_df['isDeleted']==False)
wd_pr_df = wd_pr_df.loc[wql_filter]
#Drop Null values if the employee does not have a job/sub job assignment
wd_pr_df = wd_pr_df.loc[~wd_pr_df['billableProject.descriptor'].isna()]
#Filter: Solar Jobs Only
wd_pr_df = wd_pr_df.loc[wd_pr_df['billableProject.descriptor'].str.startswith('710')]
#Extracting Employee ID (EEID) from 'Workers' column: This column also designates employees as Terminated so the substring '(Terminated)' is being removed for a clean creation of EEID from Workers
wd_pr_df['Workers'] = wd_pr_df['Workers'].str.replace('(Terminated)' , '')
wd_pr_df['EEID'] = wd_pr_df['Workers'].str.split('(').str[1]
wd_pr_df['EEID'] = wd_pr_df['EEID'].str.split(')').str[0]
#This isolates the 2 digit sub job, also accomodates the sub jobs that have a letter in them like Claims (C) and Warranty (W)
wd_pr_df['billableProject.descriptor'] = wd_pr_df['billableProject.descriptor'].str.replace('C' , '.C')
wd_pr_df['billableProject.descriptor'] = wd_pr_df['billableProject.descriptor'].str.replace('W' , '.W')
wd_pr_df['sub_job'] = wd_pr_df['billableProject.descriptor'].str.split('.').str[1]
wd_pr_df['sub_job'] = wd_pr_df['sub_job'].str.split(' ').str[0]
#Leaves behind Job ONLY
wd_pr_df['job'] = wd_pr_df.loc[:,'billableProject.descriptor'].str[0:7]
#Renaming Relevant Columns
wd_pr_df = wd_pr_df.rename({'calculatedDate':'Date','job':'Job','sub_job':'Sub_Job','amount':'Hours'},axis='columns')
wd_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
wd_pr_df_comp = wd_pr_df.loc[:,wd_cols_keep]
#excluding these employees from Workday because they will not be present in the TCP data
wd_eeid_exclude = ['6423','6525','9633','9647','9686','11780','13401','13757','14452','14523','14587','14594','15312','15358','15681','15912']
wd_pr_df_comp = wd_pr_df_comp.loc[~wd_pr_df_comp['EEID'].isin(wd_eeid_exclude)]

In [75]:
cmic_pr_df = cmic_pr_df_dev.copy()

In [76]:
#CMiC Data Manipulation
#Re-casting this column as a Date data type
cmic_pr_df["Date"] = pd.to_datetime(cmic_pr_df["Date"])
#Filter: Only Keeping Labor Hourly
cmic_pr_df = cmic_pr_df.loc[cmic_pr_df['Task'].isin(['LH','LHA'])]
#Renaming Relevant Columns
cmic_pr_df = cmic_pr_df.rename({'Employee Number':'EEID','Job Code':'Job','Sub Job Code':'Sub_Job','Regular Hours':'Hours'},axis='columns')
cmic_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
cmic_pr_df_comp = cmic_pr_df.loc[:,cmic_cols_keep]

In [77]:
#Date Filters
tcp_filter = (tcp_df_comp['Date']>=start_date)&(tcp_df_comp['Date']<=end_date)
cmic_filter = (cmic_pr_df_comp['Date']>=start_date)&(cmic_pr_df_comp['Date']<=end_date)
wd_filter = (wd_pr_df_comp['Date']>=start_date)&(wd_pr_df_comp['Date']<=end_date)

#### Grouping the Data by Different Levels

In [78]:
#Grouping by Job
tcp_grouped_job = tcp_df_comp.loc[tcp_filter].groupby(['Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_job = tcp_grouped_job.reset_index()
cmic_grouped_job = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_job = cmic_grouped_job.reset_index()
wd_grouped_job = wd_pr_df_comp.loc[wd_filter].groupby(['Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_job = wd_grouped_job.reset_index()

In [79]:
#Grouping by Job, Sub Job
tcp_grouped_subjob = tcp_df_comp.loc[tcp_filter].groupby(['Job','Sub_Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_subjob = tcp_grouped_subjob.reset_index()
cmic_grouped_subjob = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job','Sub_Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_subjob = cmic_grouped_subjob.reset_index()
wd_grouped_subjob = wd_pr_df_comp.loc[wd_filter].groupby(['Job','Sub_Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_subjob = wd_grouped_subjob.reset_index()

In [80]:
#Grouping by Job, Sub Job, EEID
tcp_grouped_ee = tcp_df_comp.loc[tcp_filter].groupby(['EEID']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_ee = tcp_grouped_ee.reset_index()
cmic_grouped_ee = cmic_pr_df_comp.loc[cmic_filter].groupby(['EEID']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_ee = cmic_grouped_ee.reset_index()
wd_grouped_ee = wd_pr_df_comp.loc[wd_filter].groupby(['EEID']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_ee = wd_grouped_ee.reset_index()

#### Unique Column Creation for the Joins

In [81]:
#Unique Columns to Join On for Sub Job Comparison
tcp_grouped_subjob['unique'] = tcp_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
cmic_grouped_subjob['unique'] = cmic_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
wd_grouped_subjob['unique'] = wd_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)

#### TCP vs. Workday Comparison

In [82]:
#TCP v. WD Job
tcp_v_wd_job = tcp_grouped_job.merge(wd_grouped_job , how = 'left' , on = 'Job' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_job['TotalHours_WD'] = tcp_v_wd_job['TotalHours_WD'].fillna(0)
tcp_v_wd_job['TotalHours_TCP'] = tcp_v_wd_job['TotalHours_TCP'].fillna(0)
tcp_v_wd_job['Hours_Variance (TCP - WD)'] = tcp_v_wd_job['TotalHours_TCP'] - tcp_v_wd_job['TotalHours_WD']
tcp_v_wd_job = tcp_v_wd_job.sort_values(['Job'  , 'Hours_Variance (TCP - WD)'] , ascending=[False,False])

In [83]:
#TCP v. WD Job, Sub Job
tcp_v_wd_subjob = tcp_grouped_subjob.merge(wd_grouped_subjob , how = 'left' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_wd_subjob['TotalHours_WD'] = tcp_v_wd_subjob['TotalHours_WD'].fillna(0)
tcp_v_wd_subjob['TotalHours_TCP'] = tcp_v_wd_subjob['TotalHours_TCP'].fillna(0)
tcp_v_wd_subjob['Hours_Variance (TCP - WD)'] = tcp_v_wd_subjob['TotalHours_TCP'] - tcp_v_wd_subjob['TotalHours_WD']
tcp_v_wd_subjob = tcp_v_wd_subjob.sort_values(['Job_TCP' , 'Sub_Job_TCP' , 'Hours_Variance (TCP - WD)'] , ascending=[False,True,False])

In [84]:
#TCP v. WD Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'left' , on = 'EEID' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_ee['TotalHours_WD'] = tcp_v_wd_ee['TotalHours_WD'].fillna(0)
tcp_v_wd_ee['TotalHours_TCP'] = tcp_v_wd_ee['TotalHours_TCP'].fillna(0)
tcp_v_wd_ee['Hours_Variance (TCP - WD)'] = tcp_v_wd_ee['TotalHours_TCP'] - tcp_v_wd_ee['TotalHours_WD']
tcp_v_wd_ee = tcp_v_wd_ee.sort_values('Hours_Variance (TCP - WD)' , ascending=False, key=abs)
#Removing Variances that are 0 - excluding EEIDs that match
tcp_v_wd_ee_xl = tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0.0]

#### TCP vs. CMiC Comparison

In [85]:
#TCP v CMiC Job
tcp_v_cmic_job = tcp_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_job['TotalHours_CMiC'] = tcp_v_cmic_job['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_job['TotalHours_TCP'] = tcp_v_cmic_job['TotalHours_TCP'].fillna(0)
tcp_v_cmic_job['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_job['TotalHours_TCP'] - tcp_v_cmic_job['TotalHours_CMiC']
tcp_v_cmic_job = tcp_v_cmic_job.sort_values(['Job' , 'Hours_Variance (TCP - CMiC)'], ascending=[False, False])

In [86]:
#TCP v CMiC Job, Sub Job
tcp_v_cmic_subjob = tcp_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_cmic_subjob['TotalHours_CMiC'] = tcp_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_subjob['TotalHours_TCP'] = tcp_v_cmic_subjob['TotalHours_TCP'].fillna(0)
tcp_v_cmic_subjob['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_subjob['TotalHours_TCP'] - tcp_v_cmic_subjob['TotalHours_CMiC']
tcp_v_cmic_subjob = tcp_v_cmic_subjob.sort_values(['Job' , 'Sub_Job', 'Hours_Variance (TCP - CMiC)'], ascending=[False, True, False])

In [87]:
#TCP v CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_ee['TotalHours_CMiC'] = tcp_v_cmic_ee['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_ee['TotalHours_TCP'] = tcp_v_cmic_ee['TotalHours_TCP'].fillna(0)
tcp_v_cmic_ee['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_ee['TotalHours_TCP'] - tcp_v_cmic_ee['TotalHours_CMiC']
tcp_v_cmic_ee = tcp_v_cmic_ee.sort_values('Hours_Variance (TCP - CMiC)', ascending=False, key=abs)
#Filter down to relevant columns only

#### Workday vs. CMiC Comparison

In [88]:
#WD v. CMiC EEID
wd_v_cmic_job = wd_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_job['TotalHours_CMiC'] = wd_v_cmic_job['TotalHours_CMiC'].fillna(0)
wd_v_cmic_job['TotalHours_WD'] = wd_v_cmic_job['TotalHours_WD'].fillna(0)
wd_v_cmic_job['Hours_Variance (WD - CMiC)'] = wd_v_cmic_job['TotalHours_WD'] - wd_v_cmic_job['TotalHours_CMiC']
wd_v_cmic_job = wd_v_cmic_job.sort_values(['Job' , 'Hours_Variance (WD - CMiC)'], ascending=[False,False])

In [89]:
#WD v. CMiC Job, Sub Job
wd_v_cmic_subjob = wd_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
wd_v_cmic_subjob['TotalHours_CMiC'] = wd_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
wd_v_cmic_subjob['TotalHours_WD'] = wd_v_cmic_subjob['TotalHours_WD'].fillna(0)
wd_v_cmic_subjob['Hours_Variance (WD - CMiC)'] = wd_v_cmic_subjob['TotalHours_WD'] - wd_v_cmic_subjob['TotalHours_CMiC']
wd_v_cmic_subjob = wd_v_cmic_subjob.sort_values(['Job_WD' , 'Sub_Job_WD' , 'Hours_Variance (WD - CMiC)'], ascending=[False,True,False])

In [90]:
#WD v. CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_ee['TotalHours_CMiC'] = wd_v_cmic_ee['TotalHours_CMiC'].fillna(0)
wd_v_cmic_ee['TotalHours_WD'] = wd_v_cmic_ee['TotalHours_WD'].fillna(0)
wd_v_cmic_ee['Hours_Variance (WD - CMiC)'] = wd_v_cmic_ee['TotalHours_WD'] - wd_v_cmic_ee['TotalHours_CMiC']
wd_v_cmic_ee = wd_v_cmic_ee.sort_values('Hours_Variance (WD - CMiC)', ascending=False , key=abs)
#Filter down to relevant columns only

In [49]:
file_path = os.getcwd() + '/excel_output'
file_path

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/jbeachy2/code/Users/JBeachy/Project_DataDomain/Payroll/git_LaborHours_Compare/excel_output'

In [50]:
#File Path in AMLS
file_name = f'Hours_Comparison_WeekEnding_{week_ending}'
#Writing to AMLS File Directory
with pd.ExcelWriter(f'{file_path}/{file_name}.xlsx') as writer:
    tcp_v_wd_job.to_excel(writer , sheet_name = 'TCPvWD_Hours_byJob' , index = False)
    tcp_v_wd_subjob.to_excel(writer , sheet_name = 'TCPvWD_Hours_bySubJob' , index = False)
    tcp_v_wd_ee_xl.to_excel(writer , sheet_name = 'TCPvWD_Hours_byEE' , index = False)
    tcp_v_cmic_job.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byJob' , index = False)
    tcp_v_cmic_subjob.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_bySubJob' , index = False)
    tcp_v_cmic_ee.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byEE' , index = False)
    wd_v_cmic_job.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byJob' , index = False)
    wd_v_cmic_subjob.to_excel(writer , sheet_name = 'WDvCMiC_Hours_bySubJob' , index = False)
    wd_v_cmic_ee.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byEE' , index = False)

In [25]:
#Blob Storage Info
storage_account_key = 'u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA=='
storage_account_name = 'mossdatalakesource'
connection_string = 'DefaultEndpointsProtocol=https;AccountName=mossdatalakesource;AccountKey=u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA==;EndpointSuffix=core.windows.net'
container_name = 'cmic'
#Defininng Blob Storage Uplpoad#
def uploadtoblobstorage(file_path,file_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container = container_name , blob = file_name)

    with open(file_path , 'rb') as data:
        blob_client.upload_blob(data)
    print(f'Uploaded {file_name}.')

In [26]:
file_name_exp = f'{file_name}.xlsx'
uploadtoblobstorage(f'{file_path}/{file_name}.xlsx', file_name_exp)

Uploaded Hours_Comparison_WeekEnding_2023-10-21.xlsx.


#### Run Statistics 

In [27]:
#### TCP vs. Workday Stats
today = date.today()
today_timechange = today - timedelta(hours=4)
today_for_text = today_timechange.strftime("%m-%d-%Y %H:%M:%S")
print("TCP vs. Workday")
print("Count of EEIDs that are showing Zero Hours Worked in Workday: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_WD'] == 0]))
print("Count of EEIDs that are showing Zero Hours Worked in TCP: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_TCP'] == 0]))
print("Count of EEIDs with a variance less than 10 hours(ABS), not including 0: ", len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']< 10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] >-10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0)]))
print("Count of EEIDs with no variance in Hours Worked: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] == 0]))
print("Count of EEIDs with variance greater than or equal to 10(ABS): ",len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']>=10)|(tcp_v_wd_ee['Hours_Variance (TCP - WD)']<=-10)]))
print(f"Ran on {today_for_text} for Week Ending {week_ending}")

TCP vs. Workday
Count of EEIDs that are showing Zero Hours Worked in Workday:  54
Count of EEIDs that are showing Zero Hours Worked in TCP:  0
Count of EEIDs with a variance less than 10 hours(ABS), not including 0:  663
Count of EEIDs with no variance in Hours Worked:  2027
Count of EEIDs with variance greater than or equal to 10(ABS):  66
Ran on 10-26-2023 09:09:32 for Week Ending 2023-10-21


In [123]:
#['timeEntryCode.id1'].notna())&(wd_pr_df['historicalStatus']!='94e8c24204d1406897f2758bd5e306d1')&(wd_pr_df['isDeleted']==False
wd_pr_df_dev.loc[wd_pr_df_dev['Workers'].str.contains('14527') , ['EmployeeID' , 'Workers', 'billableProject.descriptor' , 'billableTransaction.descriptor' , 'timeEntryCode.id1' , 'historicalStatus' , 'isDeleted']]

,EmployeeID,Workers,billableProject.descriptor,billableTransaction.descriptor,timeEntryCode.id1,historicalStatus,isDeleted
18589,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 10/02/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
18590,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 10/03/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
18591,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 10/04/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
18592,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 10/05/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
35262,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 09/25/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
35263,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 09/26/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
35264,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 09/27/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
35265,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 09/28/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
35266,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),8 Hours on 09/29/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
50544,14527,Byron Pang (14527),1412201.00 Ameresco - Kupono Solar (COW),10 Hours on 10/09/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,True


In [125]:
wd_pr_df.loc[wd_pr_df['Workers'].str.contains('14527') , ['EmployeeID' , 'Workers', 'billableProject.descriptor' , 'billableTransaction.descriptor' , 'timeEntryCode.id1' , 'historicalStatus' , 'isDeleted']]

,EmployeeID,Workers,billableProject.descriptor,billableTransaction.descriptor,timeEntryCode.id1,historicalStatus,isDeleted


In [124]:
#Look up above EEID in final Workday DF, why is he not incldued?
wd_grouped_ee.loc[wd_grouped_ee['EEID']=='14527']

,EEID,TotalHours_WD


In [92]:
wd_pr_df_comp.loc[wd_pr_df_comp['EEID']=='20761']

,EEID,Date,Job,Sub_Job,Hours
63449,20761,2023-10-10,7102023,10,2.0
63450,20761,2023-10-11,7102023,10,10.0
63451,20761,2023-10-12,7102023,10,10.0


In [93]:
wd_pr_df_dev.loc[wd_pr_df_dev['Workers'].str.contains('3943') , ['EmployeeID' , 'Workers', 'billableProject.descriptor' , 'billableTransaction.descriptor' , 'timeEntryCode.id1' , 'historicalStatus' , 'isDeleted']]

,EmployeeID,Workers,billableProject.descriptor,billableTransaction.descriptor,timeEntryCode.id1,historicalStatus,isDeleted
38598,3943,Urbano Sefuentes (3943),7102023.10 174 PG - Ho'ohana Solar (Drive Piles),10 Hours on 09/25/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
38599,3943,Urbano Sefuentes (3943),7102023.10 174 PG - Ho'ohana Solar (Drive Piles),10 Hours on 09/26/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
38608,3943,Urbano Sefuentes (3943),7102023.10 174 PG - Ho'ohana Solar (Drive Piles),10 Hours on 09/27/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
38609,3943,Urbano Sefuentes (3943),7102023.10 174 PG - Ho'ohana Solar (Drive Piles),10 Hours on 09/28/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
61127,3943,Urbano Sefuentes (3943),7102023.10 174 PG - Ho'ohana Solar (Drive Piles),10 Hours on 10/02/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
61128,3943,Urbano Sefuentes (3943),7102023.10 174 PG - Ho'ohana Solar (Drive Piles),10 Hours on 10/03/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
63336,3943,Urbano Sefuentes (3943),7102201.15 AES - West Oahu (Racking System),10 Hours on 10/09/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,True
63337,3943,Urbano Sefuentes (3943),7102201.15 AES - West Oahu (Racking System),10 Hours on 10/10/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,True
63338,3943,Urbano Sefuentes (3943),7102201.15 AES - West Oahu (Racking System),10 Hours on 10/11/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,True
63339,3943,Urbano Sefuentes (3943),7102201.15 AES - West Oahu (Racking System),10 Hours on 10/12/2023,77c9ae7eb96b1001b5ebf29ffdea0000,None,True


In [94]:
tcp_v_wd_ee.loc[tcp_v_wd_ee['EEID']=='3426']

,EEID,TotalHours_TCP,TotalHours_WD,Hours_Variance (TCP - WD)
2200,3426,39.5,0.0,39.5


In [65]:
wd_pr_df_dev.loc[wd_pr_df_dev['Workers'].str.contains('3426')]

,workdayID,EmployeeID,Employee_FullName,Workers,Employee_BillTransaction_Descriptor,billableTransaction.descriptor,Employee_BillTransactionDate,amount,dayOfTheWeek,FrequencySalary,...,workerType.descriptor,workerType.id,worktags1,worktagsAvailableForCostingAllocation1,worktagsDontMatchProjectDerivedWorktagRules,worktagsExcludingCostCenter1,worktagTypes1,worktagTypesWithInactiveValues,historicalStatus,zeroRate1
38630,e70ba2f2f4839000ca26f7f1c7fa0000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 09/25/2023,09/25/2023,10.0,Monday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
38631,e70ba2f2f4839000ca26f88b97920000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 09/26/2023,09/26/2023,10.0,Tuesday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
38648,e70ba2f2f4839000ca26f92582860000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 09/27/2023,09/27/2023,10.0,Wednesday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
38649,e70ba2f2f4839000ca26f9c091480000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 09/28/2023,09/28/2023,10.0,Thursday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
61141,7c8f1553b88990011947dfa19b9b0000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 10/02/2023,10/02/2023,10.0,Monday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
61142,7c8f1553b88990011947e03b944f0000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 10/03/2023,10/03/2023,10.0,Tuesday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
63352,dcd88da845b59002168f60ac1bdb0000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 10/09/2023,10/09/2023,10.0,Monday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
63353,dcd88da845b59002168f6146035b0000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 10/10/2023,10/10/2023,10.0,Tuesday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
63354,dcd88da845b59002168f627a2a4e0000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 10/11/2023,10/11/2023,10.0,Wednesday,Weekly Unions,...,None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...",None,None,"[{""descriptor"":""Calculation: Regular"",""id"":""36...","[{""descriptor"":""Calculation"",""id"":""4bfb6ed9c09...",None,None,False
63355,dcd88da845b59002168f63143ef80000,-,Justin Goines-Patcho,Justin Goines-Patcho (3426),10 Hours,10 Hours on 10/12/2023,10/12/2023,10.0,Thursday,Weekly Unions,...,None,None,"[{""descriptor"

In [69]:
#Why is this employee getting filtered out?
tcp_df_dev.loc[(tcp_df_dev['Employee_Number']=='3426')&(tcp_df_dev['Date']>='2023-10-15')]

,WorkSegments_Timesheet_ID,Employee_Number,First_Name,Last_Name,Date,Job_Code,Activity,Phase_Code,Task,Regular_Hours,Overtime_Hours,Pay_Rate,Employee_Classification,Department,FileName,Created_Date
381719,381720,3426,JUSTIN,GOINES-PATCHO,2023-10-17,1412201,28,48.3007.00,LH,10.0,0.0,0.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-26 09:31:26.326666666
385546,385547,3426,JUSTIN,GOINES-PATCHO,2023-10-16,1412201,28,48.3007.00,LH,10.0,0.0,0.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-26 09:31:26.326666666
397652,397653,3426,JUSTIN,GOINES-PATCHO,2023-10-19,1412201,28,48.3007.00,LH,9.5,0.0,0.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-26 09:31:26.326666666
397798,397799,3426,JUSTIN,GOINES-PATCHO,2023-10-18,1412201,28,48.3007.00,LH,10.0,0.0,0.0,,710,export-WorkSegments-Timesheet Export - Jacob.csv,2023-10-26 09:31:26.326666666
